In [3]:
import pandas as pd
import helper

In [20]:
df_genre = pd.read_csv('movies.csv')
df_genre[830:835]

,movieId,title,genres
830,1026,So Dear to My Heart (1949),Children|Drama
831,1027,Robin Hood: Prince of Thieves (1991),Adventure|Drama
832,1028,Mary Poppins (1964),Children|Comedy|Fantasy|Musical
833,1029,Dumbo (1941),Animation|Children|Drama|Musical
834,1030,Pete's Dragon (1977),Adventure|Animation|Children|Musical


In [9]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [10]:
df_movies = pd.merge(df_ratings, df_genre, on=['movieId'], how='left')
df_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [11]:
df_movies.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [12]:
def count_word(df, ref_col, liste):
    keyword_count = dict()
    for s in liste: keyword_count[s] = 0
    for liste_keywords in df[ref_col].str.split('|'):
        if type(liste_keywords) == float and pd.isnull(liste_keywords): continue
        for s in liste_keywords: 
            if pd.notnull(s): keyword_count[s] += 1
    # convert the dictionary in a list to sort the keywords  by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

In [13]:
#here we  make census of the genres:
genre_labels = set()
for s in df_movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))


In [14]:
#counting how many times each of genres occur:
keyword_occurences, dum = count_word(df_movies, 'genres', genre_labels)
keyword_occurences

[['Drama', 44752],
 ['Comedy', 38026],
 ['Action', 27056],
 ['Thriller', 25240],
 ['Adventure', 22017],
 ['Romance', 19336],
 ['Crime', 16266],
 ['Sci-Fi', 15365],
 ['Fantasy', 10657],
 ['Children', 8680],
 ['Mystery', 7625],
 ['Horror', 6790],
 ['Animation', 6170],
 ['War', 5025],
 ['Musical', 4722],
 ['IMAX', 3156],
 ['Western', 1912],
 ['Documentary', 1564],
 ['Film-Noir', 1140],
 ['(no genres listed)', 18]]

In [15]:
# Convert Genres in row to Columns
# New Column is created for every genre

tempDF = df_movies

for (key,cnt) in keyword_occurences:
    tempDF.loc[tempDF['genres'].str.contains(key), key] = 1
    tempDF[key] = tempDF[key].fillna(0)


tempDF.head()

c:\program files\python36\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,userId,movieId,rating,timestamp,title,genres,Drama,Comedy,Action,Thriller,...,Mystery,Horror,Animation,War,Musical,IMAX,Western,Documentary,Film-Noir,(no genres listed)
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Drop Unncecessary Columns

tempDF = tempDF.drop(columns=['genres'])

In [17]:
tempDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100004 entries, 0 to 100003
Data columns (total 25 columns):
userId                100004 non-null int64
movieId               100004 non-null int64
rating                100004 non-null float64
timestamp             100004 non-null int64
title                 100004 non-null object
Drama                 100004 non-null float64
Comedy                100004 non-null float64
Action                100004 non-null float64
Thriller              100004 non-null float64
Adventure             100004 non-null float64
Romance               100004 non-null float64
Crime                 100004 non-null float64
Sci-Fi                100004 non-null float64
Fantasy               100004 non-null float64
Children              100004 non-null float64
Mystery               100004 non-null float64
Horror                100004 non-null float64
Animation             100004 non-null float64
War                   100004 non-null float64
Musical               

In [18]:
tempDF.head()

,userId,movieId,rating,timestamp,title,Drama,Comedy,Action,Thriller,Adventure,...,Mystery,Horror,Animation,War,Musical,IMAX,Western,Documentary,Film-Noir,(no genres listed)
0,1,31,2.5,1260759144,Dangerous Minds (1995),1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1029,3.0,1260759179,Dumbo (1941),1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,1061,3.0,1260759182,Sleepers (1996),0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1129,2.0,1260759185,Escape from New York (1981),0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
genre_ratings = helper.get_genre_ratings(df_ratings, df_movies, ['Romance', 'Sci-Fi'], ['avg_romance_rating', 'avg_scifi_rating'])
genre_ratings.head()

     rating  rating
1      3.50    2.40
2      3.59    3.80
3      3.65    3.14
4      4.50    4.26
5      4.08    4.00
..      ...     ...
667    3.26    4.25
668    5.00     NaN
669    3.25    2.88
670    3.71    3.75
671    3.87    4.04

[671 rows x 2 columns]


,avg_romance_rating,avg_scifi_rating
1,3.50,2.40
2,3.59,3.80
3,3.65,3.14
4,4.50,4.26
5,4.08,4.00


In [20]:
genre_ratings = helper.get_genre_ratings(df_ratings, df_movies, ['Drama', 'Comedy', 'Action', 'Thriller', 'Adventure', 'Romance', 'Crime', 'Sci-Fi', 'Fantasy', 'Children', 'Mystery', 'Horror', 'Animation', 'War', 'Musical', 'IMAX', 'Western', 'Documentary', 'Film-Noir'] ,
                                         ['avg_Drama', 'avg_Comedy', 'avg_Action', 'avg_Thriller', 'avg_Adventure', 'avg_Romance', 'avg_Crime', 'avg_Sci-Fi', 'avg_Fantasy', 'avg_Children', 'avg_Mystery', 'avg_Horror', 'avg_Animation', 'avg_War', 'avg_Musical', 'avg_IMAX', 'avg_Western', 'avg_Documentary', 'avg_Film-Noir'])
genre_ratings.head()

     rating  rating  rating  rating  rating  rating  rating  rating  rating  \
1      2.57    2.00    2.80    2.83    2.17    3.50    2.50    2.40    2.12   
2      3.62    3.09    3.32    3.33    3.75    3.59    3.67    3.80    3.40   
3      3.92    3.61    3.47    3.55    3.30    3.65    3.70    3.14    3.00   
4      4.46    4.33    4.27    4.02    4.34    4.50    4.16    4.26    4.57   
5      3.81    3.98    4.00    3.62    3.82    4.08    3.45    4.00    3.75   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
667    3.71    3.88    3.27    3.50    3.57    3.26    3.67    4.25    4.00   
668    4.31    3.44    3.50    4.50    3.00    5.00    4.57     NaN    4.00   
669    3.42    3.40    3.78    3.08    3.80    3.25    3.75    2.88    3.00   
670    3.76    3.86    3.60    3.90    3.40    3.71    4.00    3.75    3.33   
671    3.78    3.81    4.00    4.05    4.09    3.87    4.05    4.04    4.41   

     rating  rating  rating  rating  rating  rating

,avg_Drama,avg_Comedy,avg_Action,avg_Thriller,avg_Adventure,avg_Romance,avg_Crime,avg_Sci-Fi,avg_Fantasy,avg_Children,avg_Mystery,avg_Horror,avg_Animation,avg_War,avg_Musical,avg_IMAX,avg_Western,avg_Documentary,avg_Film-Noir
1,2.57,2.00,2.80,2.83,2.17,3.50,2.50,2.40,2.12,2.50,NaN,3.00,2.00,2.00,3.00,NaN,3.0,NaN,NaN
2,3.62,3.09,3.32,3.33,3.75,3.59,3.67,3.80,3.40,3.50,4.00,3.50,3.67,3.80,3.75,4.00,5.0,NaN,NaN
3,3.92,3.61,3.47,3.55,3.30,3.65,3.70,3.14,3.00,2.70,3.75,3.38,2.88,4.17,2.50,2.88,4.0,3.67,NaN
4,4.46,4.33,4.27,4.02,4.34,4.50,4.16,4.26,4.57,4.63,4.09,3.94,4.71,4.75,4.83,5.00,3.0,4.00,5.0
5,3.81,3.98,4.00,3.62,3.82,4.08,3.45,4.00,3.75,3.89,3.33,3.88,3.90,4.00,4.25,4.00,NaN,3.67,NaN


In [40]:
# clustering
from sklearn.cluster import KMeans

In [45]:
X = tempDF.rating.values
print(X)

[2.5 3.  3.  ... 4.  2.5 3.5]


In [46]:
kmeans_1 = KMeans(n_clusters=2)
predictoins = kmeans_1.fit_predict(X)

ValueError: Expected 2D array, got 1D array instead:
array=[2.5 3.  3.  ... 4.  2.5 3.5].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [7]:
df[(df['rating'] > 3) | (df['userId'] == 1)]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99996,671,5991,4.5,1064245387
99997,671,5995,4.0,1066793014
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
